In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd drive/MyDrive/TML/A4

/content/drive/MyDrive/TML/A4


In [80]:
# !pip install grad-cam

In [42]:
# !pip install torch torchvision

In [81]:
# !pip install lime

TASK3

In [12]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm



# Load the pretrained model
model = models.resnet50(pretrained=True)
model.eval()

# Ensure the model runs on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 29.0MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    input_image_np = np.array(image).astype(np.float32) / 255.0
    return image, input_tensor, input_image_np

In [14]:
def predict_fn(images):
    images_tensor = torch.stack([torch.tensor(img).permute(2, 0, 1) for img in images]).to(device)
    outputs = model(images_tensor)
    probabilities = torch.nn.functional.softmax(outputs, dim=1).cpu().detach().numpy()
    return probabilities

In [15]:
import os

import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries

def apply_lime(image_path, output_path='output_t3', num_samples = 1000, top_labels = 5, num_features = 100000, segmentation_fn = None, batch_size = 10 ):
    print ("image_path",image_path, num_samples, top_labels, num_features, segmentation_fn)
    original_img, input_tensor, input_image_np = load_image(image_path)

    if output_path is not None:
      # Create the output directory if it doesn't exist
      os.makedirs(output_path, exist_ok=True)

    # Create a LIME explainer
    explainer = lime_image.LimeImageExplainer()


    num_top_labels = 1  # We only need the top label for comparison


    explanation = explainer.explain_instance(input_image_np, predict_fn, top_labels=top_labels, hide_color=None, num_samples=num_samples, num_features = num_features, segmentation_fn= segmentation_fn)
    print("explanation": explanation)

    # Get the image and mask for the top predicted class
    label = explanation.top_labels[0]
    temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=5, hide_rest=False)

    img_boundry1 = mark_boundaries(temp, mask)




    # Create a figure and axis
    fig, ax = plt.subplots()

    # Show the LIME explanation on the axis
    ax.imshow(img_boundry1)
    # Get the class name
    ax.set_title(f"LIME Explanation for {os.path.basename(image_path)} - Label {label}")
    ax.axis('off')

    # Save the plot
    image_filename = os.path.basename(image_path)
    save_path = f"lime_explanation_{image_filename}_label{label}.png" # Save as PNG

    if output_path is not None:
      output_file_path = os.path.join(output_path, save_path)
      plt.savefig(output_file_path, bbox_inches='tight')
      print(f"LIME explanation for label {label} saved to {output_file_path}")

    # Close the plot to free memory
    plt.close(fig)



    # Return the image array for plotting
    return temp, mask,img_boundry1



In [17]:


from pytorch_grad_cam import GradCAM, ScoreCAM, AblationCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

target_layers = [model.layer4[-1]]

def apply_cam(method, input_tensor, rgb_img, target_category):
    # Move input tensor to the device
    input_tensor = input_tensor.to(device)

    # Resize the rgb_img to match the input_tensor dimensions
    # input_tensor has shape (batch_size, channels, height, width)
    # rgb_img is a numpy array with shape (height, width, channels)
    # We need to resize rgb_img to (height, width, channels) where height and width match input_tensor
    resized_rgb_img = transforms.Resize((input_tensor.shape[2], input_tensor.shape[3]))(Image.fromarray((rgb_img * 255).astype(np.uint8)))
    resized_rgb_img = np.array(resized_rgb_img).astype(np.float32) / 255.0


    cam = method(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(target_category)])
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(resized_rgb_img, grayscale_cam, use_rgb=True)
    return visualization

def get_prediction(input_tensor):
    outputs = model(input_tensor)
    _, predicted = outputs.max(1)
    return predicted.item()

In [28]:
import matplotlib.pyplot as plt
import os
def compare_gradcam_lime(image_path, output_path,  num_samples = 1000, top_labels = 5, num_features = 100000, segmentation_fn = None, batch_size =10):
    # Get Grad-CAM result
    original_img, input_tensor, input_image_np = load_image(image_path)
    class_id = get_prediction(input_tensor)
    gradcam_img = apply_cam(GradCAM, input_tensor, input_image_np, class_id)

    # Generate LIME explanation and get the image array
    _,_,lime_img = apply_lime(image_path, output_path,  num_samples = num_samples, top_labels = top_labels, num_features = num_features, segmentation_fn = segmentation_fn, batch_size = batch_size)


    # Display the two results side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(gradcam_img)
    axes[0].set_title('Grad-CAM')
    axes[0].axis('off')

    axes[1].imshow(lime_img)
    axes[1].set_title('LIME')
    axes[1].axis('off')

    fig.suptitle(f"Comparison for {os.path.basename(image_path)}")
    plt.tight_layout()

    # Save the combined plot
    save_path = os.path.join(output_path, f"gradcam_lime_comparison_{os.path.basename(image_path)}")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig) # Close the figure to free memory

    print(f"Comparison plot saved to {save_path}")


def main():
    image_folder = "10_images"
    output_folder = "output_t3"
    os.makedirs(output_folder, exist_ok=True) # Create output folder if it doesn't exist

    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in sorted(image_paths):
        print(f"\nProcessing: {os.path.basename(image_path)}")
        compare_gradcam_lime(image_path, output_folder)

if __name__ == "__main__":
    main()

TASK 4

In [23]:
import time
import numpy as np
import pickle
import random
from skimage.segmentation import slic, quickshift, felzenszwalb
import os # Import os module

# List containing names of the images to create the dictionary
key_names = ['West_Highland_white_terrier', 'American_coot', 'racer', 'flamingo', 'kite', 'goldfish',
             'tiger_shark', 'vulture', 'common_iguana', 'orange']


# Heuristic function to evaluate fidelity of the explanation (based on mask center)
def evaluate_fidelity(predicted_mask, explanation):
    # Calculate the center of mass of the predicted mask and the explanation
    center_of_mask = np.mean(np.nonzero(predicted_mask), axis=1)
    # For the explanation (which might be 3D), only consider the spatial dimensions (indices 1 and 2)
    non_zero_explanation_indices = np.nonzero(explanation)
    if non_zero_explanation_indices[1].size == 0 or non_zero_explanation_indices[2].size == 0:
      return 0  # Return 0 fidelity if no non-zero pixels in explanation
    center_of_explanation = np.mean([non_zero_explanation_indices[1], non_zero_explanation_indices[2]], axis=1)


    # Calculate distance between the centers (lower distance = better fidelity)
    distance = np.linalg.norm(center_of_mask - center_of_explanation)
    fidelity_score = max(0, 1 / (1 + distance))  # Inverse of distance

    return fidelity_score

# Grid search for best parameters
def grid_search_lime(image_path, output_path = None):


    # Parameter grids for exploration
    num_samples_options = [500, 600, 700]
    top_labels_options = [3, 4, 5]
    num_features_options =  [15000, 20000, 25000]
    segmentation_fn_options = ['slic', 'quickshift', 'Felzenszwalb', 'None']
    batch_size = [5, 8, 10]

    best_fidelity_score = -float('inf') # Initialize with a very low score
    best_params = None
    best_execution_time = float('inf')


    print(f"Processing: {image_path}")


    # Iterate through parameter combinations
    for num_samples in num_samples_options:
        for top_labels in top_labels_options:
            for num_features in num_features_options:
                for segmentation_fn_name in segmentation_fn_options:
                    for bsize in batch_size:
                      print(f"Testing parameters for {image_path}: num_samples={num_samples}, top_labels={top_labels}, num_features={num_features}, segmentation_fn={segmentation_fn_name}")

                      # Map segmentation function name to the actual function
                      if segmentation_fn_name == 'slic':
                        segmentation_fn = slic
                      elif segmentation_fn_name == 'quickshift':
                        segmentation_fn = quickshift
                      elif segmentation_fn_name == 'Felzenszwalb':
                        segmentation_fn = felzenszwalb
                      else:
                        segmentation_fn = None # Handle unknown segmentation_fn

                      # Measure execution time
                      start_time = time.time()
                      # Pass the correct image_path to apply_lime
                      try:
                          explanation, predicted_mask, _ = apply_lime(image_path, output_path, num_samples, top_labels, num_features, segmentation_fn, bsize )
                          end_time = time.time()
                          execution_time = end_time - start_time

                          # Calculate heuristic fidelity score
                          fidelity_score = evaluate_fidelity(predicted_mask, explanation)
                          print(f"Fidelity Score: {fidelity_score}, Execution Time: {execution_time:.2f}s")

                          # Store the best parameters based on fidelity score and execution time
                          if fidelity_score > best_fidelity_score or (fidelity_score == best_fidelity_score and execution_time < best_execution_time):
                              best_fidelity_score = fidelity_score
                              best_execution_time = execution_time
                              best_params = {
                                  "num_samples": num_samples,
                                  "top_labels": top_labels,
                                  "num_features": num_features,
                                  "segmentation_fn": segmentation_fn,
                                  "batch_size": bsize,
                              }
                      except FileNotFoundError:
                          print(f"Skipping {image_path} due to FileNotFoundError")
                          continue


    return best_params



In [20]:
try:
  with open("explain_params.pkl", "rb") as f:
      loaded_all_params = pickle.load(f)

  print(loaded_all_params.keys())
except:
  # Initialize dictionary to store parameters for all images
  loaded_all_params = {}
  print("No parameters found")


dict_keys(['American_coot', 'West_Highland_white_terrier', 'common_iguana', 'flamingo', 'goldfish', 'kite', 'orange'])


In [30]:

# Get list of image paths
image_folder = "10_images"
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


# For each image, run the grid search and store the best parameters
for image_path in sorted(image_paths):
    # Extract image name from path
    image_name = "_".join(os.path.basename(image_path).split('_')[1:]).split('.')[0]
    print(f"\nRunning grid search for: {image_name} ({image_path})")

    if image_name not in list(loaded_all_params.keys()):

      # Run grid search for each image to find the best parameters
      # Pass the full image_path to grid_search_lime
      best_params = grid_search_lime(image_path)

      # Check if best_params is None (e.g., if image was skipped due to FileNotFoundError)
      if best_params is not None:
          # Construct the final parameters for each image
          loaded_all_params[image_name] = {
              'labels': (1,),  # Keep this as the default, adjust if needed
              'hide_color': None,
              'top_labels': best_params['top_labels'],  # Best top_labels found in grid search
              'num_features': best_params['num_features'],  # Default number of features
              'num_samples': best_params['num_samples'],  # Best num_samples found in grid search
              'batch_size': 10,  # Default batch size
              'segmentation_fn': best_params['segmentation_fn'],  # Best segmentation function found in grid search
              'distance_metric': 'cosine',  # Default distance metric
              'model_regressor': None,  # Default model regressor (you can adjust if needed)
              'random_seed': None  # Default random seed (you can set a fixed one if needed)
          }

          # Saving the dictionary to a pickle file
          with open("explain_params.pkl", "wb") as f:
              pickle.dump(loaded_all_params, f)
      else:
        print(f"No best parameters found for {image_name}. Skipping.")
    else:
      print(f"Parameters for {image_name} already exist. Skipping.")




# Output the final parameters for each image
print("\nSaved the best parameters for each image to 'explain_params.pkl'")


Running grid search for: American_coot (10_images/_American_coot.JPEG)
Parameters for American_coot already exist. Skipping.

Running grid search for: West_Highland_white_terrier (10_images/_West_Highland_white_terrier.JPEG)
Parameters for West_Highland_white_terrier already exist. Skipping.

Running grid search for: common_iguana (10_images/_common_iguana.JPEG)
Parameters for common_iguana already exist. Skipping.

Running grid search for: flamingo (10_images/_flamingo.JPEG)
Parameters for flamingo already exist. Skipping.

Running grid search for: goldfish (10_images/_goldfish.JPEG)
Parameters for goldfish already exist. Skipping.

Running grid search for: kite (10_images/_kite.JPEG)
Parameters for kite already exist. Skipping.

Running grid search for: orange (10_images/_orange.JPEG)
Parameters for orange already exist. Skipping.

Running grid search for: racer (10_images/_racer.JPEG)
Processing: 10_images/_racer.JPEG
Testing parameters for 10_images/_racer.JPEG: num_samples=1500, 

  0%|          | 0/1500 [00:00<?, ?it/s]

Fidelity Score: 0.004607443668907557, Execution Time: 28.67s
Testing parameters for 10_images/_racer.JPEG: num_samples=1500, top_labels=1, num_features=10, segmentation_fn=quickshift
image_path 10_images/_racer.JPEG 1500 1 10 <function quickshift at 0x7e8799a17d80>


  0%|          | 0/1500 [00:00<?, ?it/s]

Fidelity Score: 0.003064325687963821, Execution Time: 144.64s
Testing parameters for 10_images/_racer.JPEG: num_samples=1500, top_labels=1, num_features=15, segmentation_fn=slic
image_path 10_images/_racer.JPEG 1500 1 15 <function slic at 0x7e8799a17c40>


...

Fidelity Score: 0.00735989962896857, Execution Time: 114.71s
Testing parameters for 10_images/_vulture.JPEG: num_samples=2000, top_labels=3, num_features=15, segmentation_fn=slic
image_path 10_images/_vulture.JPEG 2000 3 15 <function slic at 0x7e8799a17c40>


  0%|          | 0/2000 [00:00<?, ?it/s]

Fidelity Score: 0.0044602878303516315, Execution Time: 41.32s
Testing parameters for 10_images/_vulture.JPEG: num_samples=2000, top_labels=3, num_features=15, segmentation_fn=quickshift
image_path 10_images/_vulture.JPEG 2000 3 15 <function quickshift at 0x7e8799a17d80>


  0%|          | 0/2000 [00:00<?, ?it/s]

Fidelity Score: 0.007773194886646041, Execution Time: 113.96s

Saved the best parameters for each image to 'explain_params.pkl'


In [79]:
import os


def create_best_output():
    image_folder = "10_images"
    output_folder = "output_t4"
    os.makedirs(output_folder, exist_ok=True) # Create output folder if it doesn't exist

    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in sorted(image_paths):
        print(f"\nProcessing: {os.path.basename(image_path)}")
        name= "_".join(image_path.split('/')[1].split('_')[1:]).split('.')[0]
        try:
          if loaded_all_params[name]['segmentation_fn'] == 'slic':
              segmentation_fn = slic
          elif loaded_all_params[name]['segmentation_fn'] == 'quickshift':
            segmentation_fn = quickshift
          else:
            segmentation_fn = None # Handle unknown segmentation_fn
          compare_gradcam_lime(image_path, output_folder, loaded_all_params[name]['num_samples'],loaded_all_params[name]['top_labels'], loaded_all_params[name]['num_features'],segmentation_fn )
        except Exception as e:
          print(f"Skipping {image_path} due to FileNotFoundError" )
          print (e)

          continue


if __name__ == "__main__":
    create_best_output()


Processing: _American_coot.JPEG
image_path 10_images/_American_coot.JPEG 600 5 50000 None


  0%|          | 0/600 [00:00<?, ?it/s]

LIME explanation for label 133 saved to output_t4/lime_explanation__American_coot.JPEG_label133.png
Comparison plot saved to output_t4/gradcam_lime_comparison__American_coot.JPEG

Processing: _West_Highland_white_terrier.JPEG
image_path 10_images/_West_Highland_white_terrier.JPEG 600 5 60000 None


  0%|          | 0/600 [00:00<?, ?it/s]

LIME explanation for label 203 saved to output_t4/lime_explanation__West_Highland_white_terrier.JPEG_label203.png
Comparison plot saved to output_t4/gradcam_lime_comparison__West_Highland_white_terrier.JPEG

Processing: _common_iguana.JPEG
image_path 10_images/_common_iguana.JPEG 800 4 60000 None


...

LIME explanation for label 751 saved to output_t4/lime_explanation__racer.JPEG_label751.png
Comparison plot saved to output_t4/gradcam_lime_comparison__racer.JPEG

Processing: _tiger_shark.JPEG
image_path 10_images/_tiger_shark.JPEG 1000 3 70000 None


  0%|          | 0/1000 [00:00<?, ?it/s]

LIME explanation for label 3 saved to output_t4/lime_explanation__tiger_shark.JPEG_label3.png
Comparison plot saved to output_t4/gradcam_lime_comparison__tiger_shark.JPEG

Processing: _vulture.JPEG
image_path 10_images/_vulture.JPEG 800 5 70000 None


  0%|          | 0/800 [00:00<?, ?it/s]

LIME explanation for label 557 saved to output_t4/lime_explanation__vulture.JPEG_label557.png
Comparison plot saved to output_t4/gradcam_lime_comparison__vulture.JPEG


In [78]:
import pickle
import requests

response = requests.post("http://34.122.51.94:9091/lime", files={"file": open("explain_params.pkl", "rb")}, headers={"token": "25359591"})
print(response.json())

{'avg_iou': 0.32885815082824027, 'avg_time': 3.3749428033828734}


In [41]:
all_p = {
    'goldfish': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,  # Fish often have a specific look, fewer labels
        'num_features': 50000,  # A smaller number of features for faster processing
        'num_samples': 300,  # Reduced samples to optimize time for small aquatic creatures
        'batch_size': 5,  # Suitable batch size for the task
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'tiger_shark': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,  # Sharks have unique features, less complex than smaller aquatic creatures
        'num_features': 60000,  # Slightly more features for detailed recognition
        'num_samples': 500,  # Sharks are larger, more features needed
        'batch_size': 10,  # Larger batch to speed up shark identification
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'kite': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,  # Birds like kites have more diversity, more labels needed
        'num_features': 70000,  # Birds require more features to identify detailed features
        'num_samples': 600,  # Slightly more samples needed for bird recognition
        'batch_size': 15,  # Larger batch size, as bird images often need more processing
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'vulture': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,  # Vultures are distinctive, more labels are beneficial
        'num_features': 70000,  # Birds with more complex features
        'num_samples': 600,  # Higher sample size for large bird recognition
        'batch_size': 15,  # Larger batch size for efficient processing
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'common_iguana': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 4,  # Lizards have a mix of features that can be more complex
        'num_features': 50000,  # Smaller reptiles need fewer features
        'num_samples': 400,  # Moderate sample size for reptiles
        'batch_size': 8,  # Reptiles can be processed with a moderate batch size
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'flamingo': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,  # Birds like flamingos have diverse features and more complex recognition
        'num_features': 75000,  # Birds need higher features for accurate classification
        'num_samples': 600,  # More samples to capture the complexity of flamingos
        'batch_size': 15,  # Higher batch size for efficient bird classification
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'American_coot': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,  # Birds again, but less complex than flamingos
        'num_features': 50000,  # Lower feature count for smaller birds
        'num_samples': 400,  # Smaller sample size for quicker processing
        'batch_size': 10,  # Standard batch size
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'West_Highland_white_terrier': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,  # Dogs are fairly distinctive, need more labels
        'num_features': 60000,  # Dogs require medium complexity feature set
        'num_samples': 500,  # Moderate sample size for dog breeds
        'batch_size': 10,  # A good batch size for dogs
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'racer': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,  # Cars have distinct features but not as many as animals
        'num_features': 40000,  # Fewer features needed to identify vehicles
        'num_samples': 300,  # Lower sample count as car images are more straightforward
        'batch_size': 8,  # A smaller batch size will do fine for cars
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'orange': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,  # Only a few key labels needed for a fruit
        'num_features': 30000,  # Fewer features, fruits are simpler objects
        'num_samples': 200,  # Fewer samples, fruit recognition is fast
        'batch_size': 5,  # Smaller batch size for fruit images
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    }
}


In [76]:
updated_params = {
    'goldfish': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,
        'num_features': 20000,
        'num_samples': 600,
        'batch_size': 5,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'tiger_shark': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,
        'num_features': 70000,
        'num_samples': 1000,
        'batch_size': 10,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'kite': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,
        'num_features': 50000,
        'num_samples': 600,
        'batch_size': 10,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'vulture': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,
        'num_features': 70000,
        'num_samples': 800,
        'batch_size': 15,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'common_iguana': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 4,
        'num_features': 60000,
        'num_samples': 800,
        'batch_size': 8,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'flamingo': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,
        'num_features': 80000,
        'num_samples': 900,
        'batch_size': 15,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'American_coot': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,
        'num_features': 50000,
        'num_samples': 600,
        'batch_size': 10,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'West_Highland_white_terrier': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 5,
        'num_features': 60000,
        'num_samples': 600,
        'batch_size': 12,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'racer': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,
        'num_features': 70000,
        'num_samples': 700,
        'batch_size': 10,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    },
    'orange': {
        'labels': (1,),
        'hide_color': None,
        'top_labels': 3,
        'num_features': 30000,
        'num_samples': 200,
        'batch_size': 5,
        'segmentation_fn': None,
        'distance_metric': 'cosine',
        'model_regressor': None,
        'random_seed': None
    }
}
